# Continuous time, periodic signal transform

### Preamble
Start by importing the Python libraries that we will require

In [ ]:
import sympy as sp
import numpy as np
import mpmath as mp
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D 
from ipywidgets import interactive, widgets
from IPython.display import display

And define a function that will return true if running in a Jupyter Notebook

In [ ]:
def is_jupyter():
    """Return true if running in a Jupyter Notebook"""
    try:
        if get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
            return True
        else:
            return False
    except: 
        return False

### User specified parameters

The following parameters can be specified. 

Parameter | Meaning
--------- | -------
<code>T_p</code> | The period of signal in the time domain (s) (e.g. 1)
<code>plot_width</code> | The width of frequency domain plot (Hz) (e.g. 20)
<code>waveform</code> | Select the waveform to use (e.g. 1)

In [ ]:
T_p = 1
plot_width = 20
waveform = 1

### Symbols
Define symbols to be used

In [ ]:
t = sp.Symbol('t')
k = sp.Symbol('k')

### The signal and its transform
First we need to define the signal and its transform.  It is a finite duration, continuous signal, so will have a continuous aperiodic transform.  Later on we will want to be able to delay the signal, so add delay as a parameter that we can alter

#### Waveform definition
Note that the list of waveforms can be added to by users

In [ ]:
# First define the set of waveforms that are available.  More can be added if desired

def signal_set(delay = 0):
    """
        Define the signals that may be used.  Additional signals can be added to this list
    """
    # Use sympy.Piecewise function Piecewise( (expr,cond), (expr,cond), ... ) to define a continuous function
    waveforms = {
        # Rectangular pulse centred on delay
        1: sp.Piecewise(
            # Rectangular pulse, with height 1 between the signal limits
            (1, (t > -0.25+delay) & (t < 0.25+delay)),
            # and elsewhere it is zero
            (0, True)
        ),
        
        # Ramp function centred on delay
        2: sp.Piecewise(
            (4*(t-delay), (t > -0.25+delay) & (t < 0.25+delay)),
            (0, True)
        ),
        
        # Triangular function centred on delay
        3: sp.Piecewise(
            # The rising section
            (4*(0.25+(t-delay)), (t < delay) & (t > -0.25+delay)),
            # and the falling section
            (4*(0.25-(t-delay)), (t > delay) & (t < 0.25+delay)),
            (0, True)
        ),
    }
    
    return waveforms

Then define the functions to access the waveforms created above

In [ ]:
def signal(waveform = 1, delay = 0):
    """
        Define the signal to be used
    """

    waveforms = signal_set(delay)
    
    return waveforms.get(waveform, 0)

def number_of_waveforms():
    """
        Count the number of waveforms defined
    """
    
    return len(signal_set())

#### Select function
The signal is generated by the function defined above, and then we create a plot of the signal.  This is done by converting the symbolic function to a set of samples that can be plotted in figures.

In [ ]:
time_equation = signal(waveform = waveform, delay = 0)

plot_limits = [-T_p*0.5, T_p*0.5]
time_function = sp.lambdify(t, time_equation, 'numpy')
time_scale = np.linspace(plot_limits[0], plot_limits[1], 400)
time_values = time_function(time_scale)

Now plot the signal waveform

In [ ]:
# Enlarge the figure, label and axis size
plt.rcParams['figure.figsize'] = 16, 8
plt.rcParams.update({'font.size': 16})

plt.figure()
ax = plt.gca()
ax.plot(time_scale, time_values)
plt.xlim(plot_limits[0], plot_limits[1])

ax.set_xlabel("Time (s)")
ax.set_ylabel('Amplitude')
ax.set_title('$x(t)$');

#### Compute the transform
Now compute the transform of the signal using:
\begin{equation}
    c_k = \frac{1}{T_p}\int_{-\frac{T_p}{2}}^{\frac{T_p}{2}} x(t) e^{-j2\pi k t/T_p} dt
\end{equation}

In the python code, the transform is computed using symbolic maths

In [ ]:
# Compute the transform of the signal
c_k = sp.integrate(time_equation*sp.exp(-sp.I*2*sp.pi*k*t/T_p)/T_p, (t, -T_p/2, T_p/2))

The result is then converted to a set of samples.

Note, that there may be divide by zero warnings generated as a result of this.  These are due to the potential difficulty in evaluating functions of the form $\frac{\sin(x)}{x}$, or other similar functions that may result from the integration.

In [ ]:
# Acquire samples of the transform
c_k_function = sp.lambdify(k, c_k, modules='numpy')
plot_limits = [-plot_width*T_p, plot_width*T_p]
frequency = np.arange(plot_limits[0], plot_limits[1]+1,1)
c_k_values = c_k_function(frequency)

### Plot the magnitude

Note that any zero values may result in an error being generated.  This should not affect the resulting figure.

In [ ]:
plt.figure()
ax = plt.gca()

yll = -50
yul = 0

plt.ylim([yll,yul])

(markerLines, stemLines, baseLines) = plt.stem(frequency,
                                               20*np.log10(abs(c_k_values)),
                                               markerfmt='ro',
                                               use_line_collection=True,
                                               bottom = yll)

left, right = plt.xlim()
plt.xlim(left+1, right-1)

ax.set_xlabel("Frequency, $k$")
ax.set_ylabel('Magnitude of (dB)')
ax.set_title('$c_k$')

# Save figure in python or ipython system
if not is_jupyter(): plt.savefig('odd_cont_time_magnitude.pdf')

### Plot the phase

In [ ]:
plt.figure()
ax = plt.gca()
yll = -np.pi*1.1
yul = np.pi*1.1

plt.ylim([yll,yul])

(markerLines, stemLines, baseLines) = plt.stem(frequency,
                                               np.angle(c_k_values),
                                               markerfmt='ro',
                                               use_line_collection=True,
                                               bottom = 0)
left, right = plt.xlim()
plt.xlim(left+1, right-1)

ax.set_yticks = ([-np.pi/2, 0, np.pi/2], ['$-\pi/2$', '0', '$\pi/2$'])
ax.set_xlabel('Frequency (rad/s)')
ax.set_ylabel('Phase (rad)')
ax.set_title('$X(\omega)$')

# Save figure in python or ipython system
if not is_jupyter(): plt.savefig('odd_cont_time_phase.pdf')

© The University of Edinburgh: Produced by D. Laurenson, School of Engineering. Initial code conversion by Xing Zixiao.